### This notebook loads and train data from the directories without having them to load on memory

In [6]:
import os
os.chdir('C:\\Projects\\ature')
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np
from keras.callbacks import ModelCheckpoint

import neuralnet.utils.data_utils as dutils
from neuralnet.utils.keras_dataset import KerasPatchesGenerator
from commons.IMAGE import Image
from PIL import Image as IMG
import utils.img_utils as imgutils
import neuralnet.utils.keras_vizual_callbacks as kvz

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Directories setup

In [7]:
### Define folders. Create if needed.
sep = os.sep
Dirs = {}
Dirs['checkpoint']   = 'assests' +sep+ 'nnet_models'
Dirs['data']      = 'data'+sep+'DRIVE'+sep+'training'
Dirs['images']    = Dirs['data'] +sep+ 'images'
Dirs['mask']      = Dirs['data'] +sep+ 'mask'
Dirs['truth']     = Dirs['data'] +sep+ '1st_manual'
Dirs['segmented'] = Dirs['data'] +sep+ 'drive_segmented'

TestDirs = {}
TestDirs['data']      = 'data'+sep+'DRIVE'+sep+'test'
TestDirs['images']    = TestDirs['data'] +sep+ 'images1'
TestDirs['mask']      = TestDirs['data'] +sep+ 'mask'
TestDirs['truth']     = TestDirs['data'] +sep+ '1st_manual'
TestDirs['segmented'] = TestDirs['data'] +sep+ 'drive_segmented'

ValidationDirs = {}
ValidationDirs['data']      = 'data'+sep+'DRIVE'+sep+'test'
ValidationDirs['images']    = ValidationDirs['data'] +sep+ 'validation'
ValidationDirs['mask']      = ValidationDirs['data'] +sep+ 'mask'
ValidationDirs['truth']     = ValidationDirs['data'] +sep+ '1st_manual'
ValidationDirs['segmented'] = ValidationDirs['data'] +sep+ 'drive_segmented'

for k, folder in Dirs.items():
    os.makedirs(folder, exist_ok=True)

def get_mask_file(file_name): 
    return file_name.split('_')[0] + '_training_mask.gif'

def get_ground_truth_file(file_name): 
    return file_name.split('_')[0] + '_manual1.gif'

def get_segmented_file(file_name):
    return file_name + '_SEG.PNG'

def get_mask_file_test(file_name): 
    return file_name.split('_')[0] + '_test_mask.gif'

classes = {
    'white': 0,
    'green': 1,
    'black': 2,
    'red': 3
}
batch_size = 120
num_classes = len(classes)
epochs = 10
patch_size = 21

### Define data sources

In [8]:
train_generator = KerasPatchesGenerator(Dirs=Dirs, patch_size=patch_size, 
                                   num_classes=num_classes,
                                   fget_mask=get_mask_file, 
                                   fget_truth=get_ground_truth_file,
                                   fget_segmented=get_segmented_file,
                                   batch_size=batch_size, transformation=imgutils.whiten_image2d)

validation_generator = KerasPatchesGenerator(Dirs=ValidationDirs, patch_size=patch_size, 
                                   num_classes=num_classes,
                                   fget_mask=get_mask_file_test, 
                                   fget_truth=get_ground_truth_file,
                                   fget_segmented=get_segmented_file,
                                   batch_size=batch_size)

test_generator = KerasPatchesGenerator(Dirs=TestDirs, patch_size=patch_size, 
                                   num_classes=num_classes,
                                   fget_mask=get_mask_file_test, 
                                   fget_truth=get_ground_truth_file,
                                   fget_segmented=get_segmented_file,
                                   batch_size=batch_size)

### 36567 batches found.
### 3650 batches found.
### 1806 batches found.


### Define model

In [9]:
model = Sequential() 

model.add(Conv2D(32, (3, 3), padding='same', input_shape=(patch_size, patch_size, 1)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [ ]:
# initiate Adam optimizer
opt = keras.optimizers.Adam(lr=0.001, decay=1e-6)

# Let's train the model using Adam
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath=Dirs['checkpoint'] + 'keras_model_checkpoint.hdf5', verbose=1, save_best_only=True)
callbacks = [checkpointer]

### Fit model from directory generators

In [ ]:
class_weights = dutils.get_class_weights(np.array(train_generator.IDs)[:,3])
model.fit_generator(
        train_generator,
        steps_per_epoch=5000,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=800, verbose=1,
        callbacks=callbacks, workers=5,
        class_weight=class_weights)

Epoch 1/10
 259/5000 [>.............................] - ETA: 23:19 - loss: 6.9470 - acc: 0.3082

In [ ]:
### If we shuffle, we won't be able to generate a segmented image.
prediction = model.predict_generator(test_generator, worker=3)
pred = np.argmax(prediction, axis=1)

In [28]:
img_obj = Image()
img_obj.load_file(data_dir=Dirs['images'], file_name='21_training.tif')
img_obj.working_arr = img_obj.image_arr[:, :, 1]

In [36]:
seg = np.zeros_like(img_obj.working_arr)

In [ ]:
np.unique(pred, return_counts=True)

In [ ]:
for ix, ID in enumerate(test_generator.IDs, 0):
    ID, i, j = ID
    try:
        if pred[ix]==classes['white'] or pred[ix]==classes['red']:
            seg[int(i),int(j)] = 255
    except:
        pass
IMG.fromarray(seg)

In [37]:
# SANITY CHECK
for ID, i, j, y in train_generator.IDs:
    if ID==0:
        seg[i,j] = 255 if y == 0 or y == 3 else seg[i,j]

In [40]:
class_weights

{0: 8.0, 1: 16.0, 2: 1.0, 3: 31.0}